In [1]:
import numpy as np
import cv2
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
cap = cv2.VideoCapture('./train/trainGray.mp4')
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

bufTemp = np.empty((frameCount, frameHeight * frameWidth), np.dtype('uint8'))

fc = 0
ret = True

while (fc < frameCount  and ret):
    ret, x = np.array(cap.read())
    x = x[:,:,0]
    x.shape = frameHeight*frameWidth
    bufTemp[fc] = x
    fc += 1

cap.release()

x = bufTemp[0, 0:frameHeight*frameWidth]
x.shape = (frameHeight, frameWidth)

In [4]:
y = np.loadtxt('./train/train.txt', np.dtype('float'))
theta = np.zeros(((frameHeight*frameWidth)+1), np.dtype('float64'))
m = frameCount-1
# alpha = [0,0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56,5.12,10.24]
# lmbda = [0,0.01,0.02,0.04,0.08,0.16,0.32,0.64,1.28,2.56,5.12,10.24]
# Alpha greater than 0.16 will overflow and not work.
# Alpha around 0.01 or lower seems to do the best.
# alpha = [0.01,0.02,0.08,0.16]
# alpha = [0.001,0.01]
alpha = [0.0001]
# lmbda = [0,0.01,0.16,0.64,5.12]
# lmbda = [0,0.64,5.12]
lmbda = [0.64]
# alpha2 = .1
# lmbda2 = .1
alpLmbTheta = np.zeros((len(alpha), len(lmbda), ((frameHeight*frameWidth)+1)), np.dtype('float64'))
alpLmbCost = np.zeros((len(alpha), len(lmbda)), np.dtype('float'))
loopCount = 2 # Was 5
costArr = np.zeros((loopCount), np.dtype('float'))

In [5]:
for idx, alp in enumerate(alpha):
    for idx2, lmbd in enumerate(lmbda):
        for i in range(loopCount):
            cost = 0
            for j in range(1,frameCount):
                firstFrame = np.array(bufTemp[j-1], dtype='int16')
                secondFrame = np.array(bufTemp[j], dtype='int16')
                x = np.empty(((frameHeight*frameWidth)+1), np.dtype('float64'))
                x[0] = 1
                x[1:(frameHeight*frameWidth)+1] = secondFrame - firstFrame
                x = np.absolute(x)
#                 print(x[0:100])
                mean = np.mean(x)
                x = x - mean
                x = x/np.std(x)
#                 print('%d, %d'%(i,j))
                grad = ((np.dot(x, alpLmbTheta[idx, idx2]) - y[j])*x)/m
#                 print((np.dot(x, alpLmbTheta[idx, idx2]) - y[j]).dtype)
#                 print(grad.dtype)
                grad[1:len(grad)] = grad[1:len(grad)] + ((lmbd/m)*alpLmbTheta[idx,idx2,1:len(alpLmbTheta[idx,idx2])])
                cost = cost + ((np.dot(x, alpLmbTheta[idx, idx2]) - y[j])**2)/2
#                 print(cost.dtype)
                alpLmbTheta[idx,idx2] = (alpLmbTheta[idx,idx2] - (alp*grad))
            
            costArr[i] = cost/(2*m)
        print(costArr)
    print('\n')

[27.10984206 12.3803256 ]




In [6]:
result = np.empty((len(alpha), len(lmbda), frameCount), np.dtype('float'))
                  
for idx in range(len(alpha)):
    for idx2 in range(len(lmbda)):
        for j in range(1,frameCount):
            firstFrame = np.array(bufTemp[j-1], dtype='int16')
            secondFrame = np.array(bufTemp[j], dtype='int16')
            x = np.empty(((frameHeight*frameWidth)+1), np.dtype('int16'))
            x[0] = 1
            x[1:(frameHeight*frameWidth)+1] = secondFrame - firstFrame
            x = np.absolute(x)
            mean = np.mean(x)
            x = x - mean
            x = x/np.std(x)
#             result[idx][idx2][j-1] = np.dot(x,theta)
            result[idx][idx2][j-1] = np.dot(x,alpLmbTheta[idx,idx2])
    
# result.tofile('./train/train2.txt',sep="\n",format="%s")

In [7]:
# cost2 = 0

for idx in range(len(alpha)):
    for idx2 in range(len(lmbda)):
        for j in range(1,frameCount):
#             cost2 = cost2 + (((result[j-1] - y[j])**2)/2)
            alpLmbCost[idx,idx2] = alpLmbCost[idx,idx2] + (((result[idx][idx2][j-1] - y[j])**2)/2)

        # cost2 = cost2/(2*m)
        # print(cost2)
        alpLmbCost[idx,idx2] = alpLmbCost[idx,idx2]/(2*m)
        print(alpLmbCost[idx,idx2])
    print('\n')

11.397383082873452




In [ ]:
[2.15687092 1.60757939]
[2.15685577 1.60758375]
[2.15662879 1.60764953]
[2.15590456 1.6078647 ]
[2.14930006 1.61019828]


[1.50191961 1.12297643]
[1.50189769 1.12298319]
[1.50156931 1.12308564]
[1.50052422 1.12342727]
[1.49117764 1.12751401]


[1.05296667 0.64122782]
[1.05283138 0.64118071]
[1.05081271 0.64052484]
[1.04448541 0.63903072]
[0.99388646 0.6533095 ]


[ 3.83219224 24.90978584]
[ 3.82889687 24.78329836]
[ 3.7801433  22.96342065]
[ 3.63229694 18.01453565]
[2.68829645 2.50867594]

In [ ]:
# Using alpha = [0.01,0.02,0.08,0.16] and lmbda = [0,0.01,0.16,0.64,5.12].
# COST:
18.0072366112331
18.00792197056106
18.018201799515296
18.05108992603151
18.35736322982058

20.667768211671344
20.668988654825743
20.6872918872337
20.745817607300108
21.287932598021055

24.358303287691385
24.362213322383123
24.42084215981376
24.608059184577392
26.279205569988093

45.89320509026542
45.7379863691767
43.549046868555735
38.06298453813079
29.215779316005758

In [ ]:
# Using alpha = [0.001,0.01] and lmbda = [0,0.64,5.12].
14.060539455234583
14.067444181535224
14.115775112531272

18.0072366112331
18.05108992603151
18.35736322982058

In [ ]:
# Using alpha = [0.0001] and lmbda = [0.64].
11.397383082873452